In [1]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import warnings
warnings.filterwarnings('ignore')
from leven import jamo_levenshtein
import pandas as pd

c:\users\bicsu\anaconda3\envs\ken\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
hanam = pd.read_excel('./results/hanma_dataset_0626.xlsx')

In [3]:
data = pd.read_excel('../00.참고자료/스마트건설_Rev.01.xlsx', names=['name','size','amount','unit','price','notes'])
data = data.iloc[1:]
data = data.dropna(subset=['price'], how='any').reset_index(drop=True)
data.iloc[26:]
data1 = data.iloc[0:4]
data2 = data.iloc[5:14]
data3 = data.iloc[15:27]
data4 = data.iloc[28:38]
data4 = data.iloc[39:49]
data4 = data.iloc[50:57]

data.head(100)

data1

# import re
# def cleanText(data):
#     #텍스트에 포함되어 있는 특수 문자 제거
#     text= re.sub('[-=+,#/\?:.a-zA-Z^%$*@\'\(\(\[\]\<\>\(\)\s]','', data)
#     return text

,name,size,amount,unit,price,notes
0,...a-1 땅깎기(토사),신설(도쟈32ton),1,㎥,720,도로공사
1,a. 토사깍기,도쟈32TON,1,㎥,720,조달청
2,흙깎기공,"토사,대규모(실적)",1,㎥,720,철도시설공단
3,(1). 흙깎기공/토사,NaN,1,㎥,720,LH공사


In [4]:
import re
def cleanText(data):
    #텍스트에 포함되어 있는 특수 문자 제거
    text= re.sub('[-=+,~a-e#*/\?:.^$@\'\(\(\[\]\<\>\(\)\s]','', data)
    return text
def fullClean(data):
    #텍스트에 포함되어 있는 특수 문자 제거
    text= re.sub('[-=+,~a-zA-Z0-9#*/\?:.^$@\'\(\(\[\]\<\>\(\)\s]','', data)
    return text

In [5]:
data = data.drop(data.index[4]).reset_index(drop=True)

In [6]:
#data.to_excel('../00.참고자료/스마트건설_Rev.01_dataset.xlsx', index=False)

In [7]:
kcc_data = pd.read_excel('../01.datas/example_kcc_dataset_v2_0626.xlsx', encoding='cp949')
kcc_data = kcc_data.fillna('')

In [8]:
kcc_data['size'] = kcc_data['size'].map(lambda x: str(x))

In [9]:
kcc_data['total'] = kcc_data['name']+kcc_data['size']+kcc_data['unit']

In [10]:
new = pd.concat([kcc_data,hanam,data], sort=False).reset_index(drop=True)

In [11]:
new = new.fillna('')

In [12]:
new['total'] = new['name']+new['size']+new['unit']

In [13]:
new.head(20)

,name,size,unit,price,total,amount,notes
0,가설사무실,"발주처,감리사무실,현장사무실 ,화장실 등",식,0,"가설사무실발주처,감리사무실,현장사무실 ,화장실 등식",,
1,가설경비실 (2.4*2.4 기준),,월,0,가설경비실 (2.4*2.4 기준)월,,
2,이동식화장실(FRP),,EA,0,이동식화장실(FRP)EA,,
3,EGI펜스,,M2,0,EGI펜스M2,,
4,스틸방음벽(자주식),,M2,0,스틸방음벽(자주식)M2,,
5,스틸방음벽(자립식),,M2,0,스틸방음벽(자립식)M2,,
6,투명방음벽(자주식),,M2,0,투명방음벽(자주식)M2,,
7,투명방음벽(자립식),,M2,0,투명방음벽(자립식)M2,,
8,PVC방음벽(자주식),,M2,0,PVC방음벽(자주식)M2,,
9,PVC방음벽(자립식),,M2,0,PVC방음벽(자립식)M2,,


In [14]:
i = new[(new.name == '공 종')].index

In [15]:
new = new.drop(i)

In [16]:
from konlpy.tag import Okt  
okt=Okt()  
token_result = []
full_re_result = []
for i in new['total']:
    token_result.append(okt.morphs(cleanText(i)))
for i in new['total']:
    full_re_result.append(okt.morphs(fullClean(i)))

In [17]:
token_result[:10]

[['가설', '사무실', '발주', '처', '감리', '사무실', '현장', '사무실', '화장실', '등식'],
 ['가설', '경비실', '2424', '기준', '월'],
 ['이동식', '화장실', 'FRPEA'],
 ['EGI', '펜스', 'M', '2'],
 ['스틸', '방음벽', '자', '주식', 'M', '2'],
 ['스틸', '방음벽', '자립', '식', 'M', '2'],
 ['투명', '방음벽', '자', '주식', 'M', '2'],
 ['투명', '방음벽', '자립', '식', 'M', '2'],
 ['PVC', '방음벽', '자', '주식', 'M', '2'],
 ['PVC', '방음벽', '자립', '식', 'M', '2']]

# 갑자기 word2vec & word vector들 평균 ㄱㄱ

In [18]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

In [45]:
%time
from gensim.models import Word2Vec

embedding_model = Word2Vec(token_result, size=100, window =2, min_count=1, workers=-1, iter=500, sg=1,seed=42)

Wall time: 0 ns


In [46]:
embedding_model.most_similar(positive=['가설','사무실'])

[('ACW858825x5160303537EA', 0.3257361352443695),
 ('명비', 0.3127039670944214),
 ('주택', 0.30389732122421265),
 ('출', 0.29133379459381104),
 ('2524150', 0.29105159640312195),
 ('장용', 0.28719568252563477),
 ('그라우팅', 0.27763810753822327),
 ('엠보', 0.26015934348106384),
 ('및', 0.2587887644767761),
 ('채널', 0.25005632638931274)]

In [47]:
token_result[-4]

['f2D5080MM개소']

In [48]:
token_result[-2]

['n어스앵커공']

In [49]:
token_result[-3]

['1천공토사M']

In [50]:
import numpy as np

In [51]:
temp = np.zeros((100,), dtype = 'float32')
aver_cnt = 0
target = token_result[-4]
print(target)
for i in target:
    temp += embedding_model[i]
    aver_cnt += 1
    avg_vec = temp / aver_cnt

['f2D5080MM개소']


## 내역 벡터를 평균이 아닌 벡터들의 합으로 계산

In [57]:
temp2 = np.zeros((100,), dtype = 'float32')
aver_cnt2 = 0
target_idx = 1259
target2 = token_result[target_idx]
target2 = ['흙','깎기'] # 내 맘대로 넣은것
print(target2)
for i in target2:
    temp2 += embedding_model[i]


['흙', '깎기']


In [58]:
best_vect = []
init_sim = 0
dap = []
dap2 = []
what = []
test = 0
for i in token_result:
    if target_idx==token_result.index(i):
        continue
    temp = np.zeros((100,), dtype = 'float32')
    aver_cnt = 0
    
    test += 1
    for j in i:
        temp += embedding_model[j] #vector 찾기
#         aver_cnt += 1
#     avg_vec = temp / aver_cnt
    sim = cos_sim(temp, temp2)
    
    dap.append([sim, i, token_result.index(i), test])
        


In [59]:
sorted_dap = sorted(dap, key=lambda x: x[0], reverse=True)[:5]

In [60]:
sorted_dap

[[0.6686973, ['1', '흙', '깎기', '공', '토사', '㎥'], 1222, 1223],
 [0.6092572, ['흙', '깎기', '공', '토사', '대규모', '실적', '㎥'], 1221, 1222],
 [0.36518982, ['강판용접Rollshp20m미만mt30ton미만TON'], 1581, 1581],
 [0.33699396,
  ['24',
   'T',
   '솔',
   '라',
   '론',
   '로이',
   '복층',
   '유리',
   '일면',
   '반',
   '강화',
   '6',
   'SRP',
   '701',
   'HS',
   '12',
   'Ar',
   '6',
   'CLM',
   '2'],
  545,
  546],
 [0.31141272,
  ['테라코타',
   '판넬',
   '시',
   '공관',
   '급',
   '18',
   'T',
   '판넬',
   '구조',
   '틀',
   '설치',
   '포함',
   'M',
   '2'],
  1036,
  1037]]

In [164]:
print(target2)

['푸기', '물', '식']


In [ ]:
new['total'][1237],new['total'][what]

In [ ]:
new.to_excel('./dataset_0701_v2_fulldatadf.xlsx', index=False)

In [ ]:
token_result[1200:]

In [ ]:
new['total'][1210:]

In [ ]:
data4

In [ ]:
data.tail(10)

In [ ]:
raw = pd.read_excel('../00.참고자료/스마트건설_Rev.01.xlsx', names=['name','size','amount','unit','price','notes'])

In [ ]:
raw1 = raw.iloc[104:108]

In [ ]:
del raw1['notes']

In [ ]:
del raw1['amount']
del raw1['price']

In [ ]:
raw1 = raw1.fillna('')

In [ ]:
raw1['total'] = raw1['name'] + raw1['size'] + raw1['unit']

In [ ]:
raw1

In [132]:
embedding_model['땅']

array([-7.9444028e-05,  3.6814737e-03,  3.8234512e-03, -3.3044554e-03,
       -1.9324798e-03,  4.4253812e-04,  3.8442779e-03,  2.6666303e-04,
        2.5910768e-03,  3.2592264e-03, -6.9985446e-04, -2.0268287e-03,
       -3.5611265e-03, -4.5207134e-03, -8.1354176e-04,  2.7450181e-03,
       -2.1818567e-03,  2.5328628e-03, -1.3061815e-03, -1.4329697e-03,
        2.5871515e-03, -4.9782166e-04, -1.7300436e-03, -1.8315738e-03,
        3.1495744e-03, -1.7635251e-03, -1.9049117e-03, -7.9796161e-04,
       -3.4806435e-03, -1.6413511e-03, -4.0138126e-03,  3.9616629e-05,
       -2.6503659e-03, -1.2361465e-03,  7.9035788e-04,  7.4815704e-05,
       -1.7726891e-03, -4.5991782e-03,  4.3866541e-03, -1.3471107e-03,
        3.6532558e-03,  3.7125095e-03, -3.7593439e-03,  3.7347684e-03,
       -1.9259480e-03, -6.5656879e-04,  4.3093297e-03, -8.3148351e-04,
        2.1070014e-03, -3.7927206e-03,  2.0758305e-03, -4.1715740e-03,
       -2.0654052e-03,  1.6283999e-03, -1.5820609e-03, -1.3131934e-03,
      

In [40]:
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer

In [43]:
from konlpy.tag import Hannanum
hannanum= Hannanum()
han_result = []
for i in new['total']:
    token_result.append(hannanum.morphs(cleanText(i)))

In [65]:
new['total'][0]

'가설사무실발주처,감리사무실,현장사무실 ,화장실 등식'

In [66]:
from soynlp.utils import DoublespaceLineCorpus
from soynlp.noun import LRNounExtractor_v2
aa = '가설사무실발주처'
noun_extractor = LRNounExtractor_v2(verbose=True)
nouns = noun_extractor.train_extract(aa)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=1260, neg=1173, common=12
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 8 from 8 sents. mem=0.680 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=8, mem=0.680 Gb
[Noun Extractor] batch prediction was completed for 0 words
[Noun Extractor] checked compounds. discovered 0 compounds
[Noun Extractor] postprocessing detaching_features : 0 -> 0
[Noun Extractor] postprocessing ignore_features : 0 -> 0
[Noun Extractor] postprocessing ignore_NJ : 0 -> 0
[Noun Extractor] 0 nouns (0 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=0.680 Gb                    
[Noun Extractor] 0.00 % eojeols are covered


In [68]:
nouns

{}